In [1]:
import re
import tweepy
import numpy as np
from tweepy import OAuthHandler
import os

In [2]:
%load_ext dotenv
%dotenv ./auth.txt

In [3]:
class TwitterClient(object):
    def __init__(self):
        try:
            self.auth = OAuthHandler(os.getenv('api_key'), os.getenv('api_secret'))
            self.auth.set_access_token(os.getenv('oauth_token'), os.getenv('oauth_token_secret'))
            self.api = tweepy.API(self.auth)
        except Tweepy.TweepError:
            print("Error: Authentication Failed")

    def get_tweets(self, query, count = 10):
        tweets = []
        try:
            fetched_tweets = self.api.search_tweets(q = query, count = count)
            for tweet in fetched_tweets:
                if tweet.text not in tweets:
                    tweets.append(tweet.text)
            return tweets
        except tweepy.TweepError as e:
            print("Error : " + str(e))
            
    def get_trending_tags(self):
        trends1 = self.api.get_place_trends(1)
        data = trends1[0] 
        trends = data['trends']
        names = []
        for trend in trends:
            if trend['name'][0]=='#':
                names.append(trend['name'])
        return names

In [4]:
api = TwitterClient()
# api.get_tweets(query = '#chess', count = 200)
api.get_trending_tags()

['#UFC269',
 '#KAI1stConcert',
 '#KAIBeyondLIVE',
 '#生シノアリスWGC',
 '#ช้อปปี้1212xแอดมิว',
 '#pazar',
 '#PowerBookII',
 '#天皇杯',
 '#HBDSuperstarRajinikanth']

In [5]:
with open('data.txt', 'w') as f:
    tweets = api.get_tweets('#FarmLaw', 20000)
    for tweet in tweets:
        f.write(tweet + "_$_")

# Read Data

In [6]:
f = open('data.txt', 'r')
tweets = f.read().split('_$_')
f.close()

# Preprocess

In [7]:
import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk import download
# download('stopwords')

def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    tweets_eng = []
    for word in tweets_clean:
        flag = True
        for i in word:
            if ord(i) >= 256:
                flag = False
                break
        if flag:
            tweets_eng.append(word)
    return tweets_eng

In [8]:
processed_tweets = [process_tweet(tweet) for tweet in tweets]
while [] in processed_tweets:
    processed_tweets.remove([])

In [9]:
len(processed_tweets)

58

In [10]:
def count_words(tweet:list, freqs:dict, wordToTweet:dict):
    for word in tweet:
        if word in freqs:
            freqs[word] += 1
            wordToTweet[word].append(tweet)
        else:
            freqs[word] = 1
            wordToTweet[word] = [tweet]
    return freqs

In [11]:
freqs = dict()
wordToTweet = dict()
for tweet in processed_tweets:
    count_words(tweet, freqs, wordToTweet)

In [12]:
freqs

{'farm': 6,
 'law': 2,
 'repeal': 2,
 'watch': 2,
 'farmersprotest': 13,
 'farmlawsrep': 5,
 'farmlaw': 26,
 'khalistani': 1,
 'farmersv': 1,
 'kisan': 6,
 'taught': 1,
 'modi': 1,
 'govt': 2,
 'lesson': 1,
 'dictatorship': 1,
 'work': 2,
 'rememb': 1,
 'pay': 2,
 'homag': 1,
 'martyr': 1,
 '..': 3,
 'red': 1,
 'salut': 1,
 'com': 1,
 '...': 4,
 'may': 1,
 'yet': 1,
 'closer': 1,
 'yesterday': 1,
 'join': 1,
 '3': 4,
 'day': 2,
 'free': 1,
 'fit': 1,
 'yoga': 1,
 'class': 1,
 'toh': 1,
 'ba': 1,
 'ek': 1,
 'jhaki': 1,
 'hai': 2,
 'hazar': 1,
 'chaubi': 1,
 'abhi': 1,
 'baaqi': 1,
 'kisanmaj': 1,
 'filucknow': 1,
 'farmer': 23,
 'hold': 1,
 'havan': 1,
 'pray': 1,
 'gen': 1,
 'rawat': 1,
 'victim': 1,
 'read': 1,
 'uttarpradesh': 1,
 'andolan': 2,
 'news': 2,
 'janta': 1,
 'tv': 1,
 'farmerprotest': 4,
 'singhu': 8,
 'border': 3,
 'delhinew': 2,
 'farmerslaw': 2,
 'farmersprot': 1,
 'jantatv': 1,
 'punjabgovern': 1,
 'farmerswon': 1,
 'delhi': 5,
 'punjab': 1,
 '700': 1,
 'centralgovern

In [13]:
freq_sorted = list(freqs.items())
freq_sorted.sort(key = lambda x : -x[1])
freq_sorted[:10]

[('farmlaw', 26),
 ('farmer', 23),
 ('farmersprotest', 13),
 ('protest', 10),
 ('singhu', 8),
 ('justin', 7),
 ('farm', 6),
 ('kisan', 6),
 ('farmlawsrep', 5),
 ('delhi', 5)]

# Clustering

In [14]:
def vectorize(tweet, alphabet):
    v = np.zeros(len(alphabet))
    for i in range(len(alphabet)):
        if alphabet[i] in tweet:
            v[i] += 1
    return v

In [15]:
def closestCluster(vector, centroids):
    closest = -1
    minDist = 2**30
    for key in centroids:
        dist = np.linalg.norm(centroids[key] - vector)
        if dist < minDist:
            minDist = dist
            closest = key
    return closest

In [16]:
def assignToCluster(clusters, vectors, centroids):
    for i in range(len(vectors)):
        c = closestCluster(vectors[i], centroids)
        clusters[c].append(i)
    return clusters

In [17]:
def kmeans(k, max_iter, vectors):
    clusters = {}
    centroids = {}
    idx = np.random.choice(len(vectors), k, replace=False)
    for i in range(k):
        clusters[i] = []
        centroids[i] = vectors[idx[i]] 
    clusters = assignToCluster(clusters, vectors, centroids)
    for _ in range(max_iter-1):
        for i in range(k):
            centroids[i] = np.zeros(len(alphabet))
            for j in clusters[i]:
                centroids[i] = centroids[i] + vectors[j]
            if clusters[i] != []:
                centroids[i] = centroids[i] / len(clusters[i])
            if len(clusters[i]):
                clusters[i].clear()
        clusters = assignToCluster(clusters, vectors, centroids)
    return clusters

In [22]:
alphabet = [i[0] for i in freq_sorted]
clusters = dict()
vectors = dict()
for i in range(len(processed_tweets)):
    vectors[i] = vectorize(processed_tweets[i], alphabet)
clusters[0] = [i for i in range(len(processed_tweets))]

In [23]:
clusters = kmeans(4, 100, vectors)

In [24]:
clusters

{0: [2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57],
 1: [0],
 2: [1],
 3: [35]}

In [21]:
print(len(alphabet))

178
